## Import modules

In [1]:
import numpy as np
import open3d as o3d
from sklearn.ensemble import RandomForestClassifier
import pdal
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from plyfile import PlyData, PlyElement
import gc
from itertools import chain
from sklearn.metrics import confusion_matrix, classification_report
import os
import joblib # Use to save model
from datetime import datetime, timezone
# from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from glob import glob
# from sklearn.model_selection import GridSearchCV
sns.set()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# ROOT_PLY = """/home/sspiegel/CapstoneData/Paris/Toronto_3D/L001.ply"""

## Load in features

In [3]:
ROOT = """/home/sspiegel/CapstoneData/Paris/training_10_classes/pickleFiles/cylinder/testing/2025_11_30T13_04_Lille2_cylinder_r_0_1414_grid_0_0283_features.npz"""
ROOT2 = """/home/sspiegel/CapstoneData/Paris/training_10_classes/pickleFiles/cylinder/testing/2025_11_30T12_36_Lille2_cylinder_r_0_2828_grid_0_0566_features.npz"""
ROOT3 = """/home/sspiegel/CapstoneData/Paris/training_10_classes/pickleFiles/cylinder/testing/2025_11_30T12_06_Lille2_cylinder_r_0_5657_grid_0_1131_features.npz"""
ROOT4 = """/home/sspiegel/CapstoneData/Paris/training_10_classes/pickleFiles/cylinder/testing/2025_11_30T11_32_Lille2_cylinder_r_1_1314_grid_0_2263_features.npz"""
ROOT5 = """/home/sspiegel/CapstoneData/Paris/training_10_classes/pickleFiles/cylinder/testing/2025_11_30T11_04_Lille2_cylinder_r_2_2627_grid_0_4525_features.npz"""
ROOT6 = """/home/sspiegel/CapstoneData/Paris/training_10_classes/pickleFiles/cylinder/testing/2025_11_30T10_20_Lille2_cylinder_r_4_5255_grid_0_9051_features.npz"""


In [4]:
fileList = [ROOT,ROOT2, ROOT3,ROOT4,ROOT5,ROOT6]

## Get computed features

In [5]:
das = [np.load(r)["array2"] for r in fileList]
das = np.hstack(das)

xyz = np.load(fileList[0])["array1"]

cls = np.load(fileList[0])["array3"]

## Load in points and labels

## Combine road markings with Ground points

In [6]:
# cls[cls==2] = 1

In [7]:
# cls[cls > 1] -= 1

## Get columns

In [8]:
cols = ["EigenSum","omnivariance","entropy","linearity","planarity","sphericity","curvature","verticality1","verticality2","HeightVariance","HeightRange","count"]

In [9]:
# cols1 = [f"""{a}_radius1""" for a in cols]
# cols2 = [f"""{a}_radius2""" for a in cols]
ff = []

for i in range(1, len(fileList) + 1):
    col = [f"""{a}_radius{i}""" for a in cols]
    ff += col
    

# allCols = cols1 + cols2
    
    
    

In [10]:
allCols = ['X', 'Y','Z'] + ff + ['label']

In [11]:
allAtrs = np.hstack((xyz,das, cls.reshape(-1, 1)))

In [12]:
del xyz, das, cls

## Create dataframe

In [13]:
total_dataframe = pd.DataFrame(allAtrs, columns=allCols).astype('float32')
total_dataframe["label"] = total_dataframe["label"].astype('int32')
for i in range(1, len(fileList) + 1):
    total_dataframe[f"""count_radius{i}"""] = total_dataframe[f"""count_radius{i}"""].astype('int32')
# total_dataframe["count_radius1"] = total_dataframe["count_radius1"].astype(int)
# total_dataframe["count_radius2"] = total_dataframe["count_radius2"].astype(int)

# total_dataframe["labelName"] = total_dataframe["label"].apply(labelPoints)

In [14]:
total_dataframe.dtypes

X                         float32
Y                         float32
Z                         float32
EigenSum_radius1          float32
omnivariance_radius1      float32
                           ...   
verticality2_radius6      float32
HeightVariance_radius6    float32
HeightRange_radius6       float32
count_radius6               int32
label                       int32
Length: 76, dtype: object

In [15]:
# total_dataframe = total_dataframe.query("label != 0")
# total_dataframe = total_dataframe.copy()

In [16]:
gc.collect()

20

In [17]:
# ss = ss[ss["count"] > 10]

In [18]:
total_dataframe["label"].max()

np.int32(9)

## Create PLY files with features (Only do if they don't already exist)

In [19]:
# for i in range(1, 7):
#     ls = [col for col in list(total_dataframe) if col.endswith(f"""radius{i}""")]
#     ls = ['X','Y','Z'] + ls + ['label']
#     partial_df = total_dataframe[ls]

#     tpsOut = []
#     for idx, tpe in partial_df.dtypes.to_dict().items():
#         if tpe == 'int64':
#             tpsOut.append((idx, 'i4'))
#         elif tpe == 'float64':
#             tpsOut.append((idx, 'f8'))
            
#     vertex_data = np.empty(allAtrs.shape[0], dtype=tpsOut)
    
#     for t in tpsOut:
#         vertex_data[t[0]] = partial_df[t[0]].values
    
        
#     el = PlyElement.describe(vertex_data, 'vertex')
    
#     # Create a PlyData object and write to a PLY file
#     # Set text=True for ASCII PLY, or text=False for binary PLY
#     PlyData([el], text=False).write(f"""/home/sspiegel/CapstoneData/Paris/Toronto_3D/PC_with_features/L002_features_radius{i}.ply""")

In [20]:
rf = joblib.load("""/home/sspiegel/CapstoneData/Paris/RF_models/2025_11_30_Trained_cylinder_RF_Lille_Height.joblib""")

In [21]:
total_dataframe.reset_index(inplace = True)

In [22]:
total_dataframe = total_dataframe.query("""label != 0""")

In [23]:
X = total_dataframe[ff]

In [24]:
y = total_dataframe["label"]
print(y.min())

# yPred = rf.predict(X)

1


In [25]:
yPred = rf.predict(X)

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:   13.3s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:   48.1s finished


In [26]:
(yPred == y).sum() / y.shape[0]

np.float64(0.9153797040699104)

In [27]:
cmNorm = confusion_matrix(y, yPred, normalize='true')

In [28]:
indx = ["ground","building","signage","bollard","trash can","barrier","pedestrian","car","vegetation"]

In [29]:
cmDF = pd.DataFrame(cmNorm, columns=indx, index = indx)

In [30]:
cmDF

,ground,building,signage,bollard,trash can,barrier,pedestrian,car,vegetation
ground,0.973480,0.001018,0.000461,0.000065,0.001476,0.000825,0.000030,0.016645,0.006001
building,0.003668,0.846036,0.004331,0.000003,0.004858,0.087627,0.000111,0.003667,0.049699
signage,0.000227,0.213082,0.697314,0.000009,0.001620,0.023311,0.000318,0.000755,0.063363
bollard,0.130036,0.005070,0.764319,0.016991,0.025075,0.003700,0.000000,0.007536,0.047273
trash can,0.071485,0.004073,0.046926,0.033706,0.438788,0.116158,0.018786,0.089563,0.180515
barrier,0.000839,0.020194,0.075632,0.000000,0.006075,0.498449,0.000219,0.025229,0.373363
pedestrian,0.000534,0.001959,0.026084,0.000000,0.012463,0.084394,0.218820,0.236891,0.418855
car,0.007503,0.000174,0.000530,0.000005,0.002133,0.027352,0.000357,0.910268,0.051679
vegetation,0.063238,0.004521,0.018025,0.000751,0.028199,0.018528,0.000245,0.038803,0.827690


In [31]:
cmDF.round(4).to_markdown()

'|            |   ground |   building |   signage |   bollard |   trash can |   barrier |   pedestrian |    car |   vegetation |\n|:-----------|---------:|-----------:|----------:|----------:|------------:|----------:|-------------:|-------:|-------------:|\n| ground     |   0.9735 |     0.001  |    0.0005 |    0.0001 |      0.0015 |    0.0008 |       0      | 0.0166 |       0.006  |\n| building   |   0.0037 |     0.846  |    0.0043 |    0      |      0.0049 |    0.0876 |       0.0001 | 0.0037 |       0.0497 |\n| signage    |   0.0002 |     0.2131 |    0.6973 |    0      |      0.0016 |    0.0233 |       0.0003 | 0.0008 |       0.0634 |\n| bollard    |   0.13   |     0.0051 |    0.7643 |    0.017  |      0.0251 |    0.0037 |       0      | 0.0075 |       0.0473 |\n| trash can  |   0.0715 |     0.0041 |    0.0469 |    0.0337 |      0.4388 |    0.1162 |       0.0188 | 0.0896 |       0.1805 |\n| barrier    |   0.0008 |     0.0202 |    0.0756 |    0      |      0.0061 |    0.4984 |       0

In [32]:
rep = classification_report(y, yPred)

In [33]:
report_dict = classification_report(y, yPred, target_names=indx, output_dict=True)

df_report = pd.DataFrame(report_dict).transpose()

# 5. Print the DataFrame
df_report.round(4)

,precision,recall,f1-score,support
ground,0.9916,0.9735,0.9824,1.204210e+07
building,0.9932,0.8460,0.9137,7.193259e+06
signage,0.5259,0.6973,0.5996,1.099060e+05
bollard,0.0224,0.0170,0.0193,7.298000e+03
trash can,0.3855,0.4388,0.4104,1.158850e+05
barrier,0.0378,0.4984,0.0703,5.481800e+04
pedestrian,0.3876,0.2188,0.2797,1.123300e+04
car,0.7169,0.9103,0.8021,7.704510e+05
vegetation,0.5922,0.8277,0.6904,9.176310e+05
accuracy,0.9154,0.9154,0.9154,9.154000e-01


In [34]:
cmDF = cmDF.round(4)
cmDF.to_csv("./results/results_ConfusionMatrix.csv")


In [35]:
df_report = df_report[["precision","recall","f1-score"]].round(4)
df_report.to_csv("./results/results_PrecisionReport.csv")

In [36]:
df_report.to_markdown()

'|              |   precision |   recall |   f1-score |\n|:-------------|------------:|---------:|-----------:|\n| ground       |      0.9916 |   0.9735 |     0.9824 |\n| building     |      0.9932 |   0.846  |     0.9137 |\n| signage      |      0.5259 |   0.6973 |     0.5996 |\n| bollard      |      0.0224 |   0.017  |     0.0193 |\n| trash can    |      0.3855 |   0.4388 |     0.4104 |\n| barrier      |      0.0378 |   0.4984 |     0.0703 |\n| pedestrian   |      0.3876 |   0.2188 |     0.2797 |\n| car          |      0.7169 |   0.9103 |     0.8021 |\n| vegetation   |      0.5922 |   0.8277 |     0.6904 |\n| accuracy     |      0.9154 |   0.9154 |     0.9154 |\n| macro avg    |      0.517  |   0.6031 |     0.5298 |\n| weighted avg |      0.9561 |   0.9154 |     0.9318 |'

In [37]:
df_report

,precision,recall,f1-score
ground,0.9916,0.9735,0.9824
building,0.9932,0.8460,0.9137
signage,0.5259,0.6973,0.5996
bollard,0.0224,0.0170,0.0193
trash can,0.3855,0.4388,0.4104
barrier,0.0378,0.4984,0.0703
pedestrian,0.3876,0.2188,0.2797
car,0.7169,0.9103,0.8021
vegetation,0.5922,0.8277,0.6904
accuracy,0.9154,0.9154,0.9154


Create Point Cloud

In [38]:
outs = total_dataframe[["X","Y","Z"]]
outs["predicted_label"] = yPred
outPly = """/home/sspiegel/CapstoneData/Paris/training_10_classes/pickleFiles/cylinder/testing/Point_Features/Lille2_pred.ply"""

tpsOut = []
for idx, tpe in outs.dtypes.to_dict().items():
    if tpe == 'int32':
        tpsOut.append((idx, 'i4'))
    elif tpe == 'float32':
        tpsOut.append((idx, 'f4'))
        
vertex_data = np.empty(outs.shape[0], dtype=tpsOut)

for t in tpsOut:
    vertex_data[t[0]] = outs[t[0]].values

    
el = PlyElement.describe(vertex_data, 'vertex')


PlyData([el], text=False).write(outPly)

/tmp/ipykernel_9193/367669122.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outs["predicted_label"] = yPred
